# <p style="background-color:#1086CE; font-family:newtimeroman; color:white; font-size:100%; text-align:center; border-radius: 15px;">[TPS-Dec] End-to-End ML Project for Beginner<img src='https://storage.googleapis.com/kaggle-competitions/kaggle/26479/logos/header.png?t=2021-04-09-00-55-58'></img></p>
Hi I'm Steve. Well come to '[TPS-2021] End-to-End ML Project for Beginner'<br>
Today I'm gonna show you how to make 'End-to-End ML Project for Beginner'<br>

## LEGO ~

# Notice
---
<img src='https://images.squarespace-cdn.com/content/v1/571fab4ad51cd4f5bee60686/ac130d5b-78e0-4e1d-a6b2-159b2911057c/notice_Asset+1%402x.png' width=600 height=200></img>


> This repository based on Hands-on-Machine Learning 2 from O’Reilly Media 


I refferenced 'Chap 2. Full Machine Learning Project<br>
If you are the beginner like me, I wish this notebook will help you to understand ML Lifecyle

* Last update
    * [x] 21.12.27.mon
    * [ ] remove features
    * [ ] fine-tuning

# TL;DR
---
<img src='https://thumbs.gfycat.com/AffectionateImmenseBronco-size_restricted.gif'></img>
- [Frame the Problem](#1)
- [Get the data](#2)
- [Explore the data](#3)
- [Prepare the data](#4)
- [Select model and train it](#5)
- [Fine-tune the model](#6)
- [Present the solution](#7)
- [Launch the ML System](#8)

[[](http://)](http://)<h3 style="color:green">If you think this notebook is helpful, upvotes would be greatly appreciated :-) </h3>

<a id='1'></a>
# 1. Frame the Problem
---
The first question is *'What is our business purpose?'*<br>


In real case, it may not just make ML model(may be for profit or else)<br>
But in our case, This is very clear. Because Kaggle tell us everything! :)


> The goal of these competitions is to provide a fun, and approachable for anyone, tabular dataset. These competitions will be great for people looking for something in between the Titanic Getting Started competition and a Featured competition

> For this competition, you will be predicting a categorical target based on a number of feature columns given in the data. The data is synthetically generated by a GAN that was trained on a the data from the Forest Cover Type Prediction. This dataset is (a) much larger, and (b) may or may not have the same relationship to the target as the original data.

And we must frame the problem.<br>

*is supervised-learning?<br>
is unsupervised-learning?<br>
is regression?<br>
is reinforcement learning?<br>
is batch inference?<br>
is online inference?<br>*

Kaggle Tell us that this competition is Supervised Learning especially Classification!<br>
(will be predicting the <code>Cover_Type</code> for each row in this file (the target integer class))<br>
Then we Frame the Problem like this

## Frame the Problem
* play Kaggle with Fun
* predicting a categorical target based on a number of feature columns given in the data

## Select evaluation metrics
> Submissions are evaluated on multi-class classification accuracy.

How kind you are :)


Because we are beginner, our problem is 'how enjoy the Kaggle'. looool!!!

*(If you guys want to more about evaluation. please refere here [[Model Evaluation] 1. Classification Metrics
](https://www.kaggle.com/leeyj0511/model-evaluation-1-classification-metrics)*

<a id='2'></a>
# 2. Get the Data
---
* Load Data

Simple, Just get the Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing
import matplotlib.pyplot as plt  # data visualization
import seaborn as sns  # data visualization
import plotly.express as px
import time

# data preprocessing
from sklearn.model_selection import StratifiedKFold  # Stratified K-Folds cross-validator
from sklearn.preprocessing import StandardScaler, LabelEncoder  # for data preprocessing
from sklearn.metrics import classification_report, accuracy_score  # for evaluation
from sklearn.model_selection import train_test_split  # for train_test_split
from scipy.stats import mode

# data transformation
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# data modeling
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

# for deep from sklearn.model_selection import cross_val_score

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.utils import to_categorical

# visualization options
from matplotlib import ticker
import time
import warnings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')

In [ ]:
print(f"train_shape: {train.shape}")
print(f"test_shape: {test.shape}")
print(f"sample_submission_shape: {sample_submission.shape}")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print(f"unique num in Id columns: {train.nunique()['Id']}")

<code>id Column</code><br>
The id column is a int64 integer column that contains unique record indicators ranging from 0 to 3,999,999. Like most Tabular Series, this is simply an identifier for the record and is likely not going to be of use for modelling purposes.

<code>Cover_Type Column</code><br>
The Cover_Type column contains the class targets we are attempting to predict. This is a multi-class classification problem. We should look first to see what class breakdown we have.


In [ ]:
# drop the 'Id' column
# because we don't need it for training
train.drop(['Id'], axis = 1, inplace = True)
test.drop(['Id'], axis = 1, inplace = True)

# define target
target = 'Cover_Type'

# explore features
features = [col for col in train.columns if col not in ['Id', target]]
print(f"len(features): {len(features)}")
print(f"features in train set: {features}")

All right!<br>
we have 4000000 train samples with 55 features

<a id='3'></a>
# 3. Explore the Data
<img src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/b/ba/Data_visualization_process_v1.png/350px-Data_visualization_process_v1.png' width=400></img>

> The next step is exploring the data!


In statistics, exploratory data analysis is an approach of analyzing data sets to summarize their main characteristics, often using statistical graphics and other data visualization methods. - wiki pedia - 

## Exploring Train Data

### Quick view of Train Data

First, define simple function<br>
Then explore data!

In [ ]:
def explore_data(data_type, data):
    print(f"number of rows in {data_type} data: {data.shape[0]}")
    print(f"number of columns in {data_type} data: {data.shape[1]}")
    print(f"number of missing values in {data_type} data: {sum(data.isna().sum())}")  # check if there is null values

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.info()  # Print a concise summary of a DataFrame.

In [ ]:
explore_data("train", train)

we have 4000000 data and have 55 columns(all int64 dtype) and no missing values

### Basic statistics of training data

In [ ]:
train.describe()

It seems that the range of values are different between the features.<br>
Scaling seems to be necessary because it is good to match the range of values between features for train a machine learning model.

## Exploring Test Data

### Quick view of Test Data

In [ ]:
test.shape

In [ ]:
test.head()

In [ ]:
test.info()  # Print a concise summary of a DataFrame.

In [ ]:
explore_data("test", test)

we have 1000000 data and have 54 columns(all int64 dtype) and no missing values

### Basic statistics of test data

In [ ]:
test.describe()

It also seems that the range of values are different between the features.<br>
So is also seems that test set need to scaling.

## Exploring Submission File

In [ ]:
sample_submission.head()

## EDA(Exploratory Data Analysis)

### Overview of Data

In [ ]:
# generate descriptive statistics using pandas describe()
train.iloc[:, :-1].describe().T.sort_values(by = 'std', ascending = False)\
                    .style.background_gradient(cmap = 'GnBu')\
                    .bar(subset = ["max"], color = '#BB0000')\
                    .bar(subset = ["mean",], color = 'green')

* "Soil_Type7" and "Soil_Type15" have only one value that is 0 for all records, So Dropping those columns.


In [ ]:
train.drop(['Soil_Type7', 'Soil_Type15'], axis = 1, inplace = True)
train.head()

### Continuous and Categorical Data Distribution

In [ ]:
features = [col for col in train.columns if col not in ['Id', 'Cover_Type', 'Soil_Type7', 'Soil_Type15']]
print(features)

In [ ]:
df = pd.concat([train[features], test[features]], axis = 0)

cat_features = [col for col in df.columns if df[col].nunique() < 25]
cont_features = [col for col in df.columns if df[col].nunique() >= 25]

# del df
print(f"Total number of features: {len(features)}")
print(f"number of categorical features: {len(cat_features)}")
print(f"number of continuous features: {len(cont_features)}")

plt.pie([len(cat_features), len(cont_features)], 
        labels = ['catetorical_features', 'continuous_features'],
       colors = ['green', 'orange'])
plt.show()

### Feature Distribution of Continuous Features

### Features Distribusion of Categorical Features

A quick way to examine the shape of data is to plot each numerical feature as a histogram. 

In [ ]:
# plot continuous features
i = 1
plt.figure()  # Create a new figure, or activate an existing figure.
fig, ax = plt.subplots(2, 5, figsize = (20, 12))

for feature in cont_features:
    plt.subplot(2, 5, i)
    sns.histplot(train[feature], color = 'blue', kde = True, bins = 100, label = 'train_'+str(feature))
    sns.histplot(test[feature], color = 'green', kde = True, bins = 100, label = 'test_'+str(feature))
    plt.xlabel(feature, fontsize=9); plt.legend()
    i += 1
    
plt.show()

* The scales of the features are very different from each other. We will look at the feature scale later.

* Many histograms have thick tails. In other words, the data is often spread out to the left and right rather than the center. These forms make it difficult to find patterns in machine learning algorithms. Let's transform these features into a more bell-shaped distribution.

In [ ]:
train[cont_features].describe()

### Feature correlation(only Continuous Features)

For continuous variables, the standard correlation coefficient between features can be easily obtained using the corr() method.

Correlation ranges from -1 to 1. A value close to 1 means a strong positive correlation (on the contrary, a value close to -1 indicates a strong negative correlation).

A correlation coefficient close to 0 means that there is no linear correlation.

In [ ]:
corr = train[cont_features + ['Cover_Type']].corr()
corr.style.background_gradient(cmap = 'coolwarm').set_precision(3)

we'll use <code>heatmap</code>

The primary purpose of heatmap is to better visualize the volumn of locations

In [ ]:
# sns.heatmap(train[cont_features + ['Cover_Type']], cmap ='RdYlGn', linewidths = 0.30, annot = True)  # got memorry error! :()

### Target distribution

In [ ]:
sns.catplot(x = 'Cover_Type', kind = "count", palette = "ch:.25", data=train)

In [ ]:
train.Cover_Type.value_counts()

* the data is unbalanced
* we have only one sample with target 5!

### Correlation matrix

### Feature correlation for all features

We'll exclude soil types as these are boolean values

In [ ]:
cols = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
       'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
       'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
       'Horizontal_Distance_To_Fire_Points', 'Wilderness_Area1',
       'Wilderness_Area2', 'Wilderness_Area3', 'Wilderness_Area4', 'Cover_Type']

In [ ]:
plt.figure(figsize = (15,10))
sns.heatmap(train[cols].corr(), vmin = -1, vmax=1, annot = True, cmap="Spectral") 

Elevation seems to have the highest negative correlation to Cover_Type although relatively weak correlation

In [ ]:
del fig, axes, i

In [ ]:
del corr

<a id='4'></a>
# 4. Prepare the Data

Now it's time to prepare the data for the machine learning algorithm. Automating this task instead of doing it manually has the following benefits:

* Easily iterate data transformations for any dataset
* You will gradually build a transformation library that you can use in future projects.
* In real systems, this function can be used to transform new data before load it into the algorithm.
* It's easy to try different data transformations, and it's convenient to see which combination works best.

### Transform Pipeline
* <code>sklearn.pipeline.Pipeline()</code> [scikit-learn docs](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

Pipeline of transforms with a final estimator.

Sequentially apply a list of transforms and a final estimator. Intermediate steps of the pipeline must be ‘transforms’, that is, they must implement fit and transform methods. The final estimator only needs to implement fit. The transformers in the pipeline can be cached using memory argument.

In [ ]:
# we already load relevant libraries at the beginning but for the sake of example, let's load libraies again!
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder  # for data preprocessing

# preprocessing for numerical data
num_transformer = SimpleImputer(strategy='median')

# preprocessinf for categorical data
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, cont_features),
        ('cat', cat_transformer, cat_features)
    ])

* <code>class sklearn.pipeline.Pipeline()</code> [scikit-learn docs](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

Applies transformers to columns of an array or pandas DataFrame.

This estimator allows different columns or column subsets of the input to be transformed separately and the features generated by each transformer will be concatenated to form a single feature space. This is useful for heterogeneous or columnar data, to combine several feature extraction mechanisms or transformations into a single transformer.

In [ ]:
# # Baseline
# rf_clf = RandomForestClassifier()

# # Bundle preprocessing and modeling code in a pipeline
# # Pipeline steps: list of tuples that are chained, in the order in which they are chained
# clf = Pipeline(steps=[
#     ('Preprocessor', preprocessor),
#     ('scaler', StandardScaler()),
#     ('model', rf_clf)
# ])

# print(cross_val_score(clf, train, X_test, cv=5).mean())

It takes really long time......... :(

I think we need to reduce the memory use!<br>
I searched some notebook
* [TPS Dec - EDA + Feat. Eng. + PseudoLab.](https://www.kaggle.com/samuelcortinhas/tps-dec-eda-feat-eng-pseudolab) 
* [TPS -DEC 2021 - LIGHTGBM - Top 200](https://www.kaggle.com/slythe/tps-dec-2021-lightgbm-top-200)

These notebooks show us how to reduce memory.

We note from train.info above that all columns are int64<br> 
We should look at the min and max of the columns to check if they need full 64bit to store the integers

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
start = time.time()

new_train_data=reduce_mem_usage(train)
new_test_data=reduce_mem_usage(test)

stop = time.time()
print(f"Time: {round((stop - start), 3)} seconds")

got memory error...!<br>
got a memory error again :< -21.12.27.am 00:24-

<a id='5'></a>
# 5. Select model and train it
Now, finally, the last step!
So far, after defining the problem, we load data and explored the data. We then split the data into a training set and a test set and created a transformation pipeline to automatically clean and prepare the data to feed into our machine learning algorithms.

You are now ready to select and train your machine learning model!

## Select model and train it
### Baseline with Random Forest Classifier

In [ ]:
# start timer
start = time.time()

# labels and features
y = new_train_data.Cover_Type
# features
X = new_train_data.drop('Cover_Type', axis=1)

# scale data
scaler = StandardScaler()
X = scaler.fit_transform(X)
test_data_preprocessed = scaler.transform(new_test_data)

# Label encoding
# encoder = LabelEncoder()
# y = encoder.fit_transform(y)

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)  # by default it will be set to 0.25
print("-" * 50)
print(f"X_train.shape: {X_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"X_test.shape: {X_test.shape}")
print(f"y_test.shape: {y_test.shape}")
print("-" * 50)


# Random Forest Classifier
rf_clf=RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

# validation
pred_valid = rf_clf.predict(X_test)
print(f"accuracy_score: {accuracy_score(y_test, pred_valid)}")
print("-" * 50)

# make predictions
rf_preds = rf_clf.predict(test_data_preprocessed)
print("rf_preds: ")
print(rf_preds[:5])
print("-" * 50)

# stop timer
stop = time.time()

print(f"Time: {round((stop - start), 3)} seconds")

In [ ]:
print("rf_preds: ")
print(rf_preds[:5])
print("-" * 50)

# stop timer
stop = time.time()

print(f"Time: {round((stop - start), 3)} seconds")

It took 30 minutes to learn<br>
I'll have to find if there's a better way

If you want to learn more about Ensemble Model.<br>
please reference these Notebooks

* [[Tutorial] Ensembel from starter to Expert!](https://www.kaggle.com/leeyj0511/tutorial-ensembel-from-starter-to-expert)
* [[Model Evaluation] 1. Classification Metrics](https://www.kaggle.com/leeyj0511/model-evaluation-1-classification-metrics)

### Validating model

In [ ]:
print(classification_report(y_test, pred_valid))

### Submission 1 - baseline

In [ ]:
sub = pd.DataFrame()
sub['Id'] = sample_submission['Id']
sub['Cover_Type'] = rf_preds
sub.head()

In [ ]:
sub.tail()

In [ ]:
sub.shape

In [ ]:
sub.to_csv('submission.csv', index=False)

[[](http://)](http://)<h1 style="color:green">Yeah! :)</h1>

now score is 0.95001 :)

# <a id='6'></a>
# 6. Fine-tune the Model

<a id='7'></a>
# 7. Present the solution

<a id='8'></a>
# 8. Launch the ML System

# Bonus
> Upgrade Model Performance!
* Referenced: [TPSDEC21-12 Eliminate cover type 4!
](https://www.kaggle.com/ambrosm/tpsdec21-12-eliminate-cover-type-4/notebook)

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred_valid)

In [ ]:
from matplotlib.ticker import PercentFormatter

# And we submit the modified predictions
sub.to_csv('submission.csv', index=False)
print(sub.Cover_Type.value_counts())

In [ ]:
sub.loc[sub.Cover_Type == 4, 'Cover_Type'] = 3
sub.Cover_Type.value_counts()

In [ ]:
from matplotlib.ticker import PercentFormatter

# And we submit the modified predictions
sub.to_csv('submission.csv', index=False)
print(sub.Cover_Type.value_counts())


# Plot the distribution of the test predictions
plt.figure(figsize=(10,3))
plt.hist(sub['Cover_Type'], bins=np.linspace(0.5, 7.5, 8), density=True, rwidth=0.7, label='Test predictions')
probes = [[1, 0.38565], [2, 0.51259], [3, 0.07817], [4, 0.00034], [6, 0.00701], [7, 0.01621]]
plt.bar([c for c, f in probes], [f for c, f in probes], label='lb frequencies', color='k', width=0.2)
plt.xticks(ticks=range(1, 8), labels=[f"{i}\n{(sub['Cover_Type'] == i).mean():.5f}" for i in range(1, 8)])
plt.xlabel('Cover_Type')
plt.ylabel('Frequency')
plt.gca().yaxis.set_major_formatter(PercentFormatter(xmax=1))
plt.legend()
plt.show()

sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.Cover_Type.value_counts()

# Recap
This will help you whenever you do ML Project.<br>
Keep in mind :)

- [x]  Frame the Problem
- [x]  Get the data
- [x]  Explore the data
- [x]  Prepare the data
- [x]  Model the data
- [x]  Fine-tune the data
- [x]  Present the solution
- [x]  Launch the ML System

# Reference
---
## Reference Notebook
* for EDA
    * [TPS-DEC] 📊EDA + Modeling🔥  [link](https://www.kaggle.com/odins0n/tps-dec-eda-modeling)
    * TPS Dec Step By Step 🏄 [link](https://www.kaggle.com/hamzaghanmi/tps-dec-step-by-step)
    * TPS Dec 21 clean EDA and prediction🔥 [link](https://www.kaggle.com/gaganmaahi224/tps-dec-21-clean-eda-and-prediction)
* for data prepare & baseline
    * [Tutorial] Ensemble from starter to Expert! [link](https://www.kaggle.com/leeyj0511/tutorial-ensembel-from-starter-to-expert)
* memory reduction'
    * TPS Dec - EDA + Feat. Eng. + PseudoLab. [link](https://www.kaggle.com/samuelcortinhas/tps-dec-eda-feat-eng-pseudolab)
* Top 200
    * TPS -DEC 2021 - LIGHTGBM - Top 200 [link](https://www.kaggle.com/slythe/tps-dec-2021-lightgbm-top-200)